In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve


def print_structure(name, obj):
    print(name)

with h5py.File('pressure_history.hdf5', 'r') as f:
    print("--- Internal File Structure ---")
    f.visititems(print_structure)
    print("-------------------------------")

# --- 1. Material & Geometry Constants ---
G, nu, nu_u, c, a, H, F = 0.75e9, 0.2, 0.44, 0.1, 1.0, 1.0, 1.0e6

# --- 2. Analytical Solution ---
def get_analytical_u(t_vec, a, G, nu, nu_u, F):
    def root_func(alpha):
        return np.tan(alpha) - ((1 - nu) / (nu_u - nu)) * alpha
    alpha1 = fsolve(root_func, 1.0)[0]
    tau = (c * t_vec) / (a**2)
    # Mandel Vertical Displacement at z=H
    u_z = (F * nu / (2 * G)) - (F * (nu_u - nu) / (G * (1 - nu_u))) * np.exp(-(alpha1**2) * tau)
    return u_z

# --- 3. Data Extraction ---
# Setting the specific filename you requested
h5_file = "pressure_history.hdf5"

try:
    with h5py.File(h5_file, 'r') as f:
        print(f"Successfully opened {h5_file}")
        
        # GEOSX internal structure check
        # Usually: nodeManager/coords and nodeManager/displacement
        if 'nodeManager' in f:
            coords_path = 'nodeManager/coords'
            disp_path = 'nodeManager/displacement'
            time_path = 'time' # This may vary based on your Event setup
        else:
            print("Structure 'nodeManager' not found. Available top-level keys:", list(f.keys()))
            # Fallback to meshio-style if that's how you converted it
            coords_path = 'mesh/points'
            disp_path = 'mesh/point_data/displacement'
            time_path = 'time'

        # Get Coordinates
        coords = f[coords_path][:]
        
        # Identify Top Center Node (Max Z, closest to X=0, Y=0)
        z_max = np.max(coords[:, 2])
        top_indices = np.where(coords[:, 2] == z_max)[0]
        dist_to_center = np.linalg.norm(coords[top_indices, :2], axis=1)
        target_node_idx = top_indices[np.argmin(dist_to_center)]
        
        print(f"Tracking Node {target_node_idx} at {coords[target_node_idx]}")

        # Extracting time-series data
        # Note: If your HDF5 contains multiple time steps in one file, 
        # displacement might be indexed by (time, node, component)
        disp_data = f[disp_path]
        
        if disp_data.ndim == 3: # (time_step, node_index, component)
            numerical_disp = disp_data[:, target_node_idx, 2]
            numerical_times = f[time_path][:] if time_path in f else np.arange(len(numerical_disp))
        else: # Single step file
            numerical_disp = [disp_data[target_node_idx, 2]]
            numerical_times = [0.0]

except FileNotFoundError:
    print(f"Error: {h5_file} not found in current directory.")
    exit()

# --- 4. Plotting ---
t_array = np.array(numerical_times)
u_analytical = get_analytical_u(t_array, a, G, nu, nu_u, F)

plt.figure(figsize=(10, 6))
plt.plot(t_array, numerical_disp, 'bo', label='GEOSX Numerical', markersize=4)
plt.plot(t_array, u_analytical, 'r-', label='Mandel Analytical', linewidth=1.5)

plt.xlabel('Time [s]')
plt.ylabel('Vertical Displacement [m]')
plt.title(f'Mandel Benchmark: Displacement from {h5_file}')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()

plt.tight_layout()
plt.savefig("mandel_plot_highres.png", dpi=600)
print("Plot saved: mandel_plot_highres.png")
plt.show()